## Runnables

There are many Runnable classes for simplifying the tasks in Langchain chains  
 - **RunnablePassthrogh**: Return Input as output as is
 - **RunnbaleLambda**: Run a function and return result of function. Lambda functions can be used here.
 - **RunnableParallel**: Run multiple chains/runnables in parallel

#### Lets Create 2 LLMs which we will use for testing

In [1]:
from langchain_ollama.chat_models import ChatOllama

llm1 = ChatOllama(
    base_url = 'http://localhost:11434',
    model = 'qwen2.5:0.5b'
)

llm2 = ChatOllama(
    base_url = 'http://localhost:11434',
    model = 'llama3.2:1b'
)

#### Runnable Passthrough

In [2]:
from langchain_core.runnables import RunnablePassthrough
r1 = RunnablePassthrough()
print(r1.invoke("Hello"))

Hello


#### Runnable Lambda

In [3]:
from langchain_core.runnables import RunnableLambda

# Create some function
def some_func(input: str) -> str:
    return f"Hello {input}"

r2 = RunnableLambda(some_func)
print(r2.invoke("Sam"))

Hello Sam


#### Runnable Parallel

In [4]:
from langchain_core.runnables import RunnableParallel

r3 = RunnableParallel(out1=r1, out2=r2)
print(r3.invoke("John"))

{'out1': 'John', 'out2': 'Hello John'}


#### Lets try to make some useful app from these learned runnables
We will give 1 topic to 2 LLMs and ask them to explain the topic to a 10 year old child.  
And then We will, evaluate which explanation is better using one of the LLM model

#### Prompt

In [5]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template="""
    User will give a topic, your task is to explain the topic to a 10 year old child. 
    The explanation must not contain complex sentence structure as it may confuse the kid. 
    Generate explanation in simple words with few examples to explain the topic clearly. 

    Topic: {topic}
    """,
    input_variables = ['topic']
)

#### First Chain from 1st LLM to generate explanation

In [6]:
from langchain_core.output_parsers import StrOutputParser
ch1 = prompt | llm1 | StrOutputParser()

# Test
ch1.invoke({"topic": "Gravity"})

"Gravity is like a strong invisible tug-of-war between your toys on Earth. It's how all our things move through space and time, just like when you pull on a toy car or play with a big ball! Imagine you're playing with your toys and you push them, they keep moving around the room instead of staying put. That's gravity in action!\n\nIf we pretend it was a really big playground called the sun, gravity is the force that makes all our balls move through space, just like how a soccer ball rolls down a hill or when you bounce on your football. It's also what keeps planets around the sun and helps us climb up to the sun in the sky!\n\nIn our galaxy, where we live, we have lots of gravity. Imagine it as a giant hug that all our friends are holding together - they move around each other and don't stop moving! It's a bit like how you're always going to be close to your mom or dad, but not everywhere.\n\nGravity is special because it can help us do cool things too. Think about the big Lego tower w

#### Second chain from other LLM model to generate explanation

In [7]:
ch2 = prompt | llm2 | StrOutputParser()

# Test
ch2.invoke({"topic": "Gravity"})

"Gravity is like a magic string that pulls everything towards each other!\n\nYou know how you're pulling on a ball and it goes down low? That's because of gravity. It's like when you throw a ball up in the air, it comes back down to your hands. Gravity is like a big hug from the Earth.\n\nImagine you have a ball, and you roll it across the floor. Where does it go first? To your hands! And then to your feet! That's because gravity is pulling everything towards each other. It's not letting us go up in the air; it's just making sure we stay on the ground.\n\nBut why do things fall down instead of staying up in the air? Well, that's also due to gravity. The Earth wants to keep everything close, so it pulls them down with its magic string. That's why you don't float off into space when you're standing still!"

#### So both the Explanations work, lets try to evaluate which explanation if better

In [8]:
eval_prompt = PromptTemplate(
    template="""
    You are an evaluator who will evaluate the 2 explanations provided by 2 LLM models and tell which explanation is better and why. 
    There is a topic for which 2 LLM models have generated an explanation for 10 year old kid. Evaluate both the explanations and 
    in response tell which explanation is better suitable for 10 year old kid and why.
    -------------------------------------------------
    Topic: {topic}
    -------------------------------------------------
    Explanation 1: {explanation_1}
    -------------------------------------------------
    Explanation 2: {explanation_2}
    -------------------------------------------------
    Evaluation:
    """,
    input_variables = ['topic', 'explanation_1', 'explanation_2']
)

In [9]:
eval_chain = RunnableParallel(explanation_1=ch1, explanation_2=ch2, topic=RunnablePassthrough()) | eval_prompt | llm2 | StrOutputParser()
ev = eval_chain.invoke({"topic": "Gravity"})
print(ev)

I'll evaluate both explanations provided by two LLM models and identify which one is better suitable for a 10-year-old kid.

**Explanation 1**

* Strengths: This explanation uses relatable examples (e.g., dropping a ball, using hands instead of the apples) to illustrate the concept of gravity.
* Weaknesses: The language is somewhat simplistic and lacks precision. For instance, "gravity is like an invisible force" might be confusing for a 10-year-old to understand.

**Explanation 2**

* Strengths:
	+ Uses simple and clear language that's easy to understand.
	+ Provides concrete examples (e.g., the balls, tug-of-war) to help illustrate the concept of gravity.
	+ Avoids using overly complex or vague terms (e.g., "invisible force").
* Weaknesses: The explanation might be a bit too basic for some 10-year-olds. It also doesn't provide any new or interesting insights into the topic.

**Evaluation**

Considering these points, I think **Explanation 2** is better suitable for a 10-year-old kid. 

#### The Evaluation is Good, But can't see what explanation was geenrated by Each LLM Model.
In order provide send the Output of Models, we can use RunParallel, Send output of LLMs as passthrough in Parallel with Evaluations

In [10]:
eval_chain_logic = eval_prompt | llm2 | StrOutputParser()

eval_chain2 = RunnableParallel(explanation_1=ch1, explanation_2=ch2, topic=RunnablePassthrough()) | \
                RunnableParallel(evaluation=eval_chain_logic, inputs=RunnablePassthrough())

ev2 = eval_chain2.invoke({"topic": "Gravity"})

print("Explanation 1: ", ev2["inputs"]["explanation_1"], end="\n\n")
print("Explanation 2: ", ev2["inputs"]["explanation_2"], end="\n\n")
print("Final Evaluation: ", ev2["evaluation"])

Explanation 1:  Okay, little one! Let's talk about something really cool - gravity! Imagine you're playing with a big box of building blocks. Now, if I put some smaller boxes on top of those big blocks, where would that make the whole stack fall down? It would land at the bottom right in our world.

Gravity is like a giant invisible hand pulling everything towards the center. It's not always easy to understand! You can't see it unless you're standing near an object or someone telling you about gravity. But when you're outside, you might notice it while playing with toys or riding bikes!

Let's imagine you're at home and suddenly feel a soft weight on your feet. That's gravity! You know how you get taller as you grow up? Gravity keeps us in the air because of this big invisible hand.

So, remember: gravity is always there, making things move and falling down to help you stay safe. Isn't that cool?

Is there anything else you'd like me to explain about gravity, little one?

Explanation 2

#### Great but there is just 1 problem. Since input is passed through, it becomes 1 dictionary.
Lets try to extract the keys from inputs into different keys using Lambda

In [12]:
eval_chain_logic = eval_prompt | llm2 | StrOutputParser()

eval_chain2 = RunnableParallel(explanation_1=ch1, explanation_2=ch2, topic=RunnablePassthrough()) | \
                RunnableParallel(evaluation=eval_chain_logic, 
                                 explanation_1=RunnablePassthrough() | RunnableLambda(lambda x: x['explanation_1']),
                                 explanation_2=RunnablePassthrough() | RunnableLambda(lambda x: x['explanation_2']),
                                )

ev2 = eval_chain2.invoke({"topic": "Gravity"})

# So now we can access all keys directly
print("Explanation 1: ", ev2["explanation_1"], end="\n\n")
print("Explanation 2: ", ev2["explanation_2"], end="\n\n")
print("Final Evaluation: ", ev2["evaluation"])

Explanation 1:  Gravity is like an invisible force that pulls things together, much like how a strong wind can blow a leaf away.
For example, if you dropped a ball from the top of the house, it would fall towards the ground because of gravity. Similarly, when we jump up high in the air, our bodies get heavier and it helps us keep going.

Explanation 2:  Gravity is something that keeps us on the ground and holds things close together.

You know how you can throw a ball up in the air, and it comes back down? That's because of gravity. It's like a magic string that pulls everything towards each other.

Imagine you have a big magnet, and you put it near some paper clips. The magnet will pull all the paper clips towards itself. Gravity is kind of like that, but with planets instead of magnets.

The Earth has too much mass (that means it has many particles stuck together), so gravity pulls everything towards it really strongly. That's why we don't float off into space when we're standing on 